In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

In [ ]:
file_path = './pdfs/human-agent-collab-problem-solving.pdf'

docs = PyPDFLoader(file_path).load_and_split()

len(docs)

In [ ]:
docs[0:5]

In [ ]:
vectordb = Chroma.from_documents(
    documents=docs,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings()
)

retriever = vectordb.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    'retrieve_info_from_paper',
    'Search and return information about a paper.')

tools = [retriever_tool]

In [ ]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]



In [ ]:
def agent(state: AgentState):
    messages = state['messages']
    model = ChatOpenAI(temperature=0, streaming=True, model='gpt-4o')
    model = model.bind_tools(tools)
    response = model.invoke(messages)
    
    return {'messages': [response]}
